**Objetivo**

En el marco del desarrollo de la clase *Data Analytics*,  a continuación se presenta el desarrollo del análisis de datos (básico) para describir el comportamiento del suicidio en sur america. Se examirá el comportamiento de los suicidios desde las variable pais, año, generación y población.


**Descripción Base de datos inicial**



In [ ]:
import numpy as np                # linear algebra
import pandas as pd               # data frames
import seaborn as sns             # visualizations
import matplotlib.pyplot as plt   # visualizations
import scipy.stats                # statistics
from sklearn import preprocessing
import os

print(os.listdir("../input"))
df = (pd.read_csv("../input/master.csv"))
print(df.head())
print(df.info())
print(df.shape)


La base incial contiene 12 categorias, de las cuales 6 son númericas (int64 y float64). Así los datos se encuentran agrupados por país , año, sexo , rango de años del suicida y generación.
También obervamos que la categoria HDI no contiene la misma cantidad de datos que las demás y que el gdp_for_year tiene al menos un dato que no es numérico. Posteriormente los modificaremos.


In [ ]:
print (df.head())


**Adaptación de la base de datos**

Para tratar la categoría de producto interno bruto como un campo numérico  quitamos las comas. No usaremos las columnas HDI for year pues no esta completa para todos los datos cómo se oberva en la siguiente gráfica. Adicional eliminaremos el campos country-year.

Entonces se eliminan las categorias mencionadas, y se adapta la variable gdp en dólares a una variable númerica.

In [ ]:
fig = plt.figure(figsize=(20,3))
sns.heatmap(df.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')


In [ ]:
df.columns = ['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicidesper100kpop', 'country-year', 'HDI for year',
       'gdp_for_year', 'gdp_per_capita', 'generation']



In [ ]:
df['gdp_for_year'] = df['gdp_for_year'].str.replace(',','').astype(int)
dfmod = df.drop(['HDI for year','country-year'],axis=1)

In [ ]:
print(dfmod.head())
print(dfmod.info())
print(dfmod.shape)
print(dfmod.dtypes)


**Exploración de los datos (cuantitativos y cualitativos)**

Observemos la descripción de los datos cuantitativos

In [ ]:
dfmod.describe()

El menor año con datos es 1985, y el mayor año 2016. Existen población (discriminada por sexo generación y edad) en un determinado año dónde
no hubo ningún suicidio y en una única caraterización 22.338 suicidios.

In [ ]:
dfmod.describe(include=['O'])

Existen 101 paises con datos de suicidios, dos clasificaciones de sexo (male, female), y seis caracterizaciones de edad y generación.
Adelante encontraremos la descripción de dichas variables contadas por números de suicidios.

In [ ]:
test=dfmod.groupby(["generation"])["suicides_no"].sum().reset_index().rename(columns={'generation':'generation', '':'suicide_no'})
#test=test.reset_index
#test.columns = ['generation', 'death_numbers']
test.head()
#np.shape(test)

sns.barplot(x="suicides_no", y="generation", data=test)
plt.show()

La variable generación contiene las categorias Boomers, GI Generation (generación perdida), Generation X, Generation Z, Millenials y Silent (WW II y posguerra). Existe una mayor incidencia de suiciduis en la generación Boomers, seguido por la generación X. 

In [ ]:

test2=dfmod.groupby(["sex"])["suicides_no"].sum().reset_index().rename(columns={'sex':'sex', '':'suicide_no'})

test2.head()
#np.shape(test)

sns.barplot(x="sex", y="suicides_no", data=test2)
plt.show()


In [ ]:
test3=dfmod.groupby(["age"])["suicides_no"].sum().reset_index().rename(columns={'age':'age', '':'suicide_no'})
test3.head()
#np.shape(test)

sns.barplot(x="suicides_no", y="age", data=test3)
plt.show()

Las 6 categoria de edad (age) son 15-24 años, 25-34 años, 35-54 años, 5-14 años, 55-74 años y mayores de 75. En los rangos 35-54 y 55-74 es dónde se presentan más suicidios, sin embargo son los rangos que abarcan más años lo cual podría ayudar a la cantidad de suicidios 
que recogen.

In [ ]:
suicidebycountry = dfmod[['country','suicides_no']].groupby(['country']).sum()
suicidebycountry.plot(kind='bar', figsize=(40,10), fontsize=25)

Finalmente obervamos el total de suicidios por país, dónde Rusia, Estados Unidos y Japón son los pases con más suicidios, sin embargo cabe resaltar de nuevo que los paises no tienen datos en todos los años lo que puede afectar las conclusiones y la decisión de qué pais tiene más o menos suicidios.

**Suicidios en América del Sur**

   A continuación se hará un análisis del comportamiento de los suicidios en américa del sur

In [ ]:
suramerica = ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Paraguay', 'Uruguay']
df_ne = df[df['country'].isin(suramerica)]
ax = df_ne.groupby(['country', 'year'])['suicides_no'].sum().unstack('country').plot(figsize=(10, 10))
ax.set_title('Número de suicidios en Ámerica del Sur', fontsize=20)
ax.legend(fontsize=15)
ax.set_xlabel('Year', fontsize=20)
ax.set_ylabel('suicides_no', fontsize=20, color='green')
plt.show()

In [ ]:
#sns.pairplot(df)
#plt.show()
# https://python-graph-gallery.com/all-charts/

En primer lugar vemos que la base no posee datos para todos los paises de Suramérica. Tenemos datos de Argentina, Brazil
Chile, Colombia, Ecuador, Uruguay y Paraguay. De lejos, Brazil es el país de Suramérica con más número de suicidios y 
Paraguay tiene el número menor, sin embargo veamos ahora el comportamiento relativo (suicidios por cada 100 mil habitantes)

In [ ]:
suramericarel = ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Paraguay', 'Uruguay']
df_ne = df[df['country'].isin(suramericarel)]
ax = df_ne.groupby(['country', 'year'])['suicidesper100kpop'].sum().unstack('country').plot(figsize=(10, 10))
ax.set_title('Número de suicidios en Ámerica del Sur', fontsize=20)
ax.legend(fontsize=15)
ax.set_xlabel('Year', fontsize=20)
ax.set_ylabel('suicidesper100kpop', fontsize=20, color='green')
plt.show()

Ahora vemos que,aunque Paraguay sigue siendo el de menos suicidios relativos por 100 habitantes, pero Uruguay pasa a  ocupar el primer lugar. Esta visión es más adecuada ya que en un pais con más población seguramente habrán más suicidios, por eso al verlo de forma absoluta no es concluyente.

Para el desarrollo del notebook use cómo guia otros notebook

https://www.kaggle.com/snaily16/suicide-rates-analysis-beginner-level
https://www.kaggle.com/ahnsangil/suicide-rates-analysis
https://www.kaggle.com/donatasbag/suicides-in-1985-2016

y adicional usé https://python-graph-gallery.com/all-charts/